<a href="https://colab.research.google.com/github/qwertyqwertyqwertyqwerty123/colab_projects/blob/main/mnk_grad_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Импорт библиотек
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# Загрузка csv файла из google drive
from google.colab import drive
drive.mount('/content/drive')

dir_path = "/content/drive/My Drive/"

# Загрузка данных
data = pd.read_csv(dir_path + '3.csv', header=None)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
X = data.iloc[:, :-1].values  # факторные признаки
y = data.iloc[:, -1].values   # результативный признак

In [ ]:
# Добавление столбца единиц для свободного члена w0
X = np.column_stack((np.ones(X.shape[0]), X))

In [ ]:
# Разделим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Нормализация данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Добавляем столбец единиц к нормализованным данным
X_train_scaled = np.hstack((np.ones((X_train_scaled.shape[0], 1)), X_train_scaled))
X_test_scaled = np.hstack((np.ones((X_test_scaled.shape[0], 1)), X_test_scaled))

In [ ]:
# МНК
# 1. Регрессия с использованием МНК (без библиотечных функций)
def linear_regression_mnk(X, y):
    XTX = X.T @ X
    if np.linalg.det(XTX) == 0:
        XTX_inv = np.linalg.pinv(XTX)
    else:
        XTX_inv = np.linalg.inv(XTX)
    theta = XTX_inv @ X.T @ y
    return theta

In [ ]:
# 2. Регрессия с использованием градиентного спуска с проверкой на NaN и масштабированием
def linear_regression_gd(X, y, lr=0.000001, epochs=10000):
    m, n = X.shape
    theta = np.zeros(n)
    for _ in range(epochs):
        gradient = (1/m) * (X.T @ (X @ theta - y))
        theta -= lr * gradient
        if np.any(np.isnan(theta)):
            raise ValueError("Градиентный спуск не сошелся: найдены NaN значения. Попробуйте уменьшить lr.")
    return theta

In [ ]:
# 3. Вычисление R^2 (коэффициента детерминации)
def r2_score(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - ss_res / ss_tot

In [ ]:
# Построение моделей
# Методом наименьших квадратов
theta_mnk = linear_regression_mnk(X_train, y_train)
# Методом градиентного спуска
theta_gd = linear_regression_gd(X_train_scaled, y_train, lr=0.001, epochs=20000)

# Предсказания на основе тестовых данных
y_pred_mnk = X_test @ theta_mnk
y_pred_gd = X_test_scaled @ theta_gd

# Оценка качества моделей (R^2)
r2_mnk = r2_score(y_test, y_pred_mnk)
r2_gd = r2_score(y_test, y_pred_gd)

print("Коэффициенты регрессии (МНК):", theta_mnk)
print("Коэффициенты регрессии (Градиентный спуск):", theta_gd)
print(f"R^2 (МНК): {r2_mnk:.4f}")
print(f"R^2 (Градиентный спуск): {r2_gd:.4f}")

Коэффициенты регрессии (МНК): [-5.60625504e+03  3.34374526e+00  1.14782847e+02]
Коэффициенты регрессии (Градиентный спуск): [11157.2522516      0.          9654.96054023   672.24665623]
R^2 (МНК): 0.9106
R^2 (Градиентный спуск): 0.9106
